This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-testing-tutorials/anomaly_detection).

## Anomaly Detection

Wallaroo provides multiple methods of analytical analysis to verify that the data received and generated during an inference is accurate.  This tutorial will demonstrate how to use anomaly detection to track the outputs from a sample model to verify that the model is outputting acceptable results.

Anomaly detection allows organizations to set validation parameters in a pipeline.  A **validation** is added to a pipeline to test data based on an expression, and flag any inferences where the validation failed to the [InferenceResult](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/#inferenceresult-object) object and the pipeline logs.

This tutorial will follow this process in setting up a validation to a pipeline and examining the results:

1. Create a workspace and upload the sample model.
1. Establish a pipeline and add the model as a step.
1. Add a validation to the pipeline.
1. Perform inferences and display anomalies through the `InferenceResult` object and the pipeline log files.

This tutorial provides the following:

* Housing model: `./models/housingprice.onnx` - a pretrained model used to determine standard home prices.
* Test Data:  `./data` - sample data.

This demonstration assumes that a Wallaroo instance has been installed.

## Steps

### Import libraries

The first step is to import the libraries needed for this notebook.

In [2]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import json

from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

### Connect to Wallaroo Instance

The following command will create a connection to the Wallaroo instance and store it in the variable `wl`.

In [3]:
# Client connection from local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "YOUR PREFIX"
wallarooSuffix = "YOUR SUFFIX"

wallarooPrefix = "sparkly-apple-3026"
wallarooSuffix = "wallaroo.community"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

In [4]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

True


### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

In [6]:
workspace_name = 'houseprice'
main_pipeline_name = 'housepricingpipeline'
ab_pipeline_name = 'housepriceabtesting'
shadow_pipeline_name = 'housepriceshadowtesting'
model_name_control = 'housingcontrol'
model_name_challenger_01 = 'housingchallenger01'
model_name_challenger_02 = 'housingchallenger02'
model_file_name_control = './models/rf_model.onnx'
model_file_name_challenger_01 = './models/xgb_model.onnx'
model_file_name_challenger_02 = './models/gbr_model.onnx'

In [7]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [8]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'houseprice', 'id': 152, 'archived': False, 'created_by': '138bd7e6-4dc8-4dc1-a760-c9e721ef3c37', 'created_at': '2023-03-13T20:43:33.923135+00:00', 'models': [{'name': 'anomalyhousingcontrol', 'versions': 6, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 14, 46, 29, 85602, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 13, 20, 43, 36, 960968, tzinfo=tzutc())}, {'name': 'anomalyhousingchallenger', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 14, 46, 30, 142854, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 13, 22, 25, 27, 993199, tzinfo=tzutc())}, {'name': 'housingcontrol', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 15, 31, 46, 695685, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 14, 15, 31, 46, 695685, tzinfo=tzutc())}, {'name': 'housingchallenger01', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 15, 31,

## Upload The Models

For our example, we will upload three models, all pre-trained to determine housing prices based on various variables.

The assumption is that we have a pipeline deployed that has been determining house prices for some time using 

* `rf_model.onnx`: The champion model that has been used in this environment for some time.
* `xgb_model.onnx` and `models/gbr_model.onnx`: Rival models that will be tested against the champion.

We will upload all three models into our workspace, then deploy a pipeline with just `rf_model.onnx` and perform some sample inferences.

In [73]:
housing_model_control = wl.upload_model(model_name_control, model_file_name_control).configure()
housing_model_challenger01 = wl.upload_model(model_name_challenger_01, model_file_name_challenger_01).configure()
housing_model_challenger02 = wl.upload_model(model_name_challenger_02, model_file_name_challenger_02).configure()

### Build the Control Sample Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.  We'll call it `housepricingpipeline`, set `rf_model.onnx` as a pipeline step as set in the variable declarations above, and run a few sample inferences.

In [74]:
mainpipeline = wl.build_pipeline(main_pipeline_name).add_model_step(housing_model_control).deploy()

### Testing

We'll use two inferences as a quick sample test - one that has a house that should be determined around $700k, the other with a house determined to be around $1.5 million.

In [75]:
if arrowEnabled is True:
    normal_input = pd.DataFrame.from_records({"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]})
else:
    normal_input = {"tensor": [[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]]}
result = mainpipeline.infer(normal_input)
display(result)

,time,in.tensor,out.variable,check_failures
0,2023-03-14 16:51:29.889,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0


In [12]:
if arrowEnabled is True:
    large_house_input = pd.DataFrame.from_records({'tensor': [[4.0, 3.0, 3710.0, 20000.0, 2.0, 0.0, 2.0, 5.0, 10.0, 2760.0, 950.0, 47.6696, -122.261, 3970.0, 20000.0, 79.0, 0.0, 0.0]]})
else:
    large_house_input = {'tensor': [[4.0, 3.0, 3710.0, 20000.0, 2.0, 0.0, 2.0, 5.0, 10.0, 2760.0, 950.0, 47.6696, -122.261, 3970.0, 20000.0, 79.0, 0.0, 0.0]]}

large_house_result = mainpipeline.infer(large_house_input)
display(large_house_result)

,time,in.tensor,out.variable,check_failures
0,2023-03-14 15:52:02.720,"[4.0, 3.0, 3710.0, 20000.0, 2.0, 0.0, 2.0, 5.0, 10.0, 2760.0, 950.0, 47.6696, -122.261, 3970.0, 20000.0, 79.0, 0.0, 0.0]",[1514079.4],0


As one last sample, we'll run through roughly 1,000 inferences at once and show a few of the results.

In [13]:
if arrowEnabled is True:
    large_inference_result = mainpipeline.infer_from_file("./data/xtest-1k.df.json")
    display(large_inference_result.head(5))
else:
    large_inference_result = mainpipeline.infer_from_file("./data/xtest-1k.json")
    display(large_inference_result)

,time,in.tensor,out.variable,check_failures
0,2023-03-14 15:52:03.334,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.75],0
1,2023-03-14 15:52:03.334,"[2.0, 2.5, 2170.0, 6361.0, 1.0, 0.0, 2.0, 3.0, 8.0, 2170.0, 0.0, 47.7109, -122.017, 2310.0, 7419.0, 6.0, 0.0, 0.0]",[615094.56],0
2,2023-03-14 15:52:03.334,"[3.0, 2.5, 1300.0, 812.0, 2.0, 0.0, 0.0, 3.0, 8.0, 880.0, 420.0, 47.5893, -122.317, 1300.0, 824.0, 6.0, 0.0, 0.0]",[448627.72],0
3,2023-03-14 15:52:03.334,"[4.0, 2.5, 2500.0, 8540.0, 2.0, 0.0, 0.0, 3.0, 9.0, 2500.0, 0.0, 47.5759, -121.994, 2560.0, 8475.0, 24.0, 0.0, 0.0]",[758714.2],0
4,2023-03-14 15:52:03.334,"[3.0, 1.75, 2200.0, 11520.0, 1.0, 0.0, 0.0, 4.0, 7.0, 2200.0, 0.0, 47.7659, -122.341, 1690.0, 8038.0, 62.0, 0.0, 0.0]",[513264.7],0


## A/B Testing

Now that we have our main pipeline set, let's experiment with other models using two methods:  A/B Testing, and Shadow Deployment.  We'll use A/B testing first.

A/B Testing takes one champion or control model and pits it against one or more challenger models.  In this case, the inference data is split between the champion and control models based on a ratio we provide.  For our example, we will be using a random split, so there is a random weighted chance whether inference data is submitted to a champion or the challenger models.  Results are shown in the same `out.variable` as for the champion model, and we can determine which model received the input data based on the `out._model_split` column.

Shadow deploy works much the same way, only **all** inference data is submitted to **all** models equally, with only the results of the champion model displayed in the `out.variable` column.  We'll demonstrate that in a later example.

### Define The A/B Testing Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

We'll use the pipeline `housepriceabtesting` for our sample pipeline, then deploy it.

In [76]:
abpipeline = (wl.build_pipeline(ab_pipeline_name)
            .add_random_split([(2, housing_model_control), (1, housing_model_challenger01 )], "session_id")).deploy()

### A/B Testing Single Inference

Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [79]:
results = []
if arrowEnabled is True:
    # use dataframe JSON files
    for x in range(5):
        result = abpipeline.infer_from_file("data/xtest-1.df.json")
        display(result.loc[:,["out._model_split", "out.variable"]])        
else:
    # use Wallaroo JSON files
    for x in range(5):
        results.append(abpipeline.infer_from_file("data/xtest-1.json"))
    for result in results:
        print(result[0].model())
        print(result[0].data())

,out._model_split,out.variable
0,"[{""name"":""housingchallenger01"",""version"":""db7ae792-9308-4de1-90ed-e233e6c28c94"",""sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}]",[659806.0]


,out._model_split,out.variable
0,"[{""name"":""housingcontrol"",""version"":""a9c0ab70-6534-41ab-a502-9e2234a8da12"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}]",[718013.7]


,out._model_split,out.variable
0,"[{""name"":""housingchallenger01"",""version"":""db7ae792-9308-4de1-90ed-e233e6c28c94"",""sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}]",[659806.0]


,out._model_split,out.variable
0,"[{""name"":""housingcontrol"",""version"":""a9c0ab70-6534-41ab-a502-9e2234a8da12"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}]",[718013.7]


,out._model_split,out.variable
0,"[{""name"":""housingchallenger01"",""version"":""db7ae792-9308-4de1-90ed-e233e6c28c94"",""sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}]",[659806.0]


### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

The sample code will be shown below.  Since this splits a single inference request into 1,000 pieces and submits them serially, this can take a few minutes to run.  We'll just go to the results for our demonstration, but users can use the code below as a sample exercise.

In [80]:

# if arrowEnabled is True:
#     responses = pd.DataFrame()
#     #Read in the test data as one dataframe
#     test_data = pd.read_json('data/xtest-1k.df.json')
#     # For each row, submit that row as a separate dataframe
#     # Add the results to the responses array
#     for index, row in test_data.head(1000).iterrows():
#         responses = responses.append(abpipeline.infer(row.to_frame('tensor').reset_index(drop=True)))
#     #now get our responses for each row
#     # each r is a dataframe, then get the result from out.split into json and get the model name
#     l = [json.loads(row['out._model_split'][0])['name'] for index, row in responses.iterrows()]
#     df = pd.DataFrame({'model': l})
#     display(df.model.value_counts())
# else:
#     l = []
#     responses =[]
#     from data import test_data
#     for nth in range(1000):
#         responses.extend(abpipeline.infer(test_data.data[nth]))
#     l = [r.raw['model_name'] for r in responses]
#     df = pd.DataFrame({'model': l})
#     display(df.model.value_counts())

housingcontrol         702
housingchallenger01    298
Name: model, dtype: int64

In [83]:
# load the inference result data from a/b testing
responses = pd.read_json('./data/abtestingresults.df.json', orient="records")
l = [json.loads(row['out._model_split'][0])['name'] for index, row in responses.iterrows()]
df = pd.DataFrame({'model': l})
display(df.model.value_counts())

housingcontrol         702
housingchallenger01    298
Name: model, dtype: int64

### Compare A/B Testing Results

### Test Challenger

Now we have run a large amount of data we can compare the results.

For this experiment we are looking for a significant change average predicted price between the two models.

In [84]:
control_count = df.model.value_counts()['housingcontrol']
challenger_count = df.model.value_counts()['housingchallenger01']

control_sum = 0
challenger_sum = 0

if arrowEnabled is True:
    for index, row in responses2.iterrows():
        if json.loads(row['out._model_split'][0])['name'] == 'housingcontrol':
            control_sum += row['out.variable'][0]
        else:
            challenger_sum += row['out.variable'][0]
else:
    for r in responses2:
        if r.raw['model_name'] == "housing-control":
            control_count += 1
            control_sum += r.raw['outputs'][0]['Float']['data'][0]
        else:
            challenger_count +=1
            challenger_sum += r.raw['outputs'][0]['Float']['data'][0]

print("control mean price prediction: " + str(control_sum/control_count))
print("challenger mean price prediction: " + str(challenger_sum/challenger_count))

control mean price prediction: 523799.4716239315
challenger mean price prediction: 576765.7087248319


### View A/B Logs

Logs can be viewed with the Pipeline method `logs()`.  For this example, only the first 5 logs will be shown.  For Arrow enabled environments, the model type can be found in the column `out._model_split`.

In [85]:
if arrowEnabled is True:
    logs = abpipeline.logs().head(5)
else:
    logs = abpipeline.logs(limit=5)
display(logs)

,time,in.tensor,out._model_split,out.variable,check_failures
0,2023-03-14 16:57:17.212,"[3.0, 1.0, 1600.0, 3200.0, 1.5, 0.0, 0.0, 3.0, 7.0, 1600.0, 0.0, 47.653, -122.331, 1860.0, 3420.0, 105.0, 0.0, 0.0]","[{""name"":""housingchallenger01"",""version"":""db7ae792-9308-4de1-90ed-e233e6c28c94"",""sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}]",[651217.1],0
1,2023-03-14 16:57:17.518,"[2.0, 1.0, 980.0, 5110.0, 1.0, 0.0, 0.0, 4.0, 7.0, 780.0, 200.0, 47.7002, -122.314, 1430.0, 5110.0, 75.0, 0.0, 0.0]","[{""name"":""housingchallenger01"",""version"":""db7ae792-9308-4de1-90ed-e233e6c28c94"",""sha"":""31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c""}]",[411594.66],0
2,2023-03-14 16:57:17.798,"[3.0, 2.0, 2100.0, 31550.0, 1.0, 0.0, 0.0, 3.0, 8.0, 2100.0, 0.0, 47.6907, -121.917, 1860.0, 18452.0, 4.0, 0.0, 0.0]","[{""name"":""housingcontrol"",""version"":""a9c0ab70-6534-41ab-a502-9e2234a8da12"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}]",[641162.2],0
3,2023-03-14 16:57:18.066,"[3.0, 2.25, 1460.0, 1353.0, 2.0, 0.0, 0.0, 3.0, 8.0, 1050.0, 410.0, 47.5774, -122.412, 1690.0, 3776.0, 2.0, 0.0, 0.0]","[{""name"":""housingcontrol"",""version"":""a9c0ab70-6534-41ab-a502-9e2234a8da12"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}]",[499651.56],0
4,2023-03-14 16:57:18.346,"[3.0, 1.75, 1360.0, 18123.0, 1.0, 0.0, 0.0, 3.0, 8.0, 1360.0, 0.0, 47.4716, -121.756, 1570.0, 16817.0, 31.0, 0.0, 0.0]","[{""name"":""housingcontrol"",""version"":""a9c0ab70-6534-41ab-a502-9e2234a8da12"",""sha"":""e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6""}]",[252192.9],0


### Undeploy The A/B Pipeline

With the example complete, we undeploy the pipeline to return the resources back to the Wallaroo instance.

In [86]:
abpipeline.undeploy()

name,housepriceabtesting
created,2023-03-14 15:52:10.722286+00:00
last_updated,2023-03-14 16:51:31.324771+00:00
deployed,False
tags,
versions,"977ed457-e0da-438d-bfa1-5a0c92c5eadb, 04edcc6a-a178-407e-b699-69b4f4804ade, 00460662-1d18-49a7-a7b4-0846673e9a5b, 51f995de-89d2-49f8-aae2-b803a2331ed9"
steps,housingcontrol


## Shadow Deploy

The other method for comparing models is Shadow Deploy.  In Shadow Deploy, the pipeline step is added with the `add_shadow_deploy` method, with the champion model listed first, then an array of challenger models after.  **All** inference data is fed to **all** models, with the champion results displayed in the `out.variable` column, and the shadow results in the format `out_{model name}.variable`.  For example, since we named our challenger models `housingchallenger01` and `housingchallenger02`, the columns `out_housingchallenger01.variable` and `out_housingchallenger02.variable` have the shadow deployed model results.

Here, we'll create a new pipeline called `housepriceshadowtesting`, then add `rf_model.onnx` as our champion, and models `xgb_model.onnx` and `gbr_model.onnx` as the challengers.  We'll deploy the pipeline and prepare it for sample inferences.

In [88]:
shadow_pipeline = wl.build_pipeline(shadow_pipeline_name).add_shadow_deploy(housing_model_control, [housing_model_challenger01, housing_model_challenger02]).deploy()

### Shadow Deploy Sample Inference

We'll now use our same sample data for an inference to our shadow deployed pipeline.

In [90]:
if arrowEnabled is True:
    shadow_result = shadow_pipeline.infer_from_file('./data/xtest-1.df.json')
else:
    shadow_result = shadow_pipeline.infer_from_file('./data/xtest-1.json')
display(shadow_result)

,time,in.tensor,out.variable,check_failures,out_housingchallenger01.variable,out_housingchallenger02.variable
0,2023-03-14 17:05:10.211,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[718013.7],0,[659806.0],[704901.9]


### Shadow Deploy Batch Inference

We can also perform batch inferences with shadow deployed pipelines.  Here we'll pass 1,000 inference requests at once, then display the results.

In [97]:
if arrowEnabled is True:
    shadow_results = shadow_pipeline.infer_from_file('./data/xtest-1k.df.json')
    display(shadow_results.loc[:,['out.variable','out_housingchallenger01.variable','out_housingchallenger02.variable']])
else:
    shadow_results = shadow_pipeline.infer_from_file('./data/xtest-1k.json')
    display(shadow_results)

,out.variable,out_housingchallenger01.variable,out_housingchallenger02.variable
0,[718013.75],[659806.0],[704901.9]
1,[615094.56],[732883.5],[695994.44]
2,[448627.72],[419508.84],[416164.8]
3,[758714.2],[634028.8],[655277.2]
4,[513264.7],[427209.44],[426854.66]
...,...,...,...
995,[827411.0],[743487.94],[787589.25]
996,[441960.38],[381577.16],[411258.3]
997,[1060847.5],[1520770.0],[1491293.8]
998,[706823.56],[663008.75],[594914.2]


### Shadow Deploy Logs

Shadow deployed results are also displayed in the log files.  For Arrow enabled Wallaroo instances, it's just the pipeline `logs` method.  For Arrow disabled environments, the command `logs_shadow_deploy()` displays the shadow deployed model information.

In [101]:
if arrowEnabled is True:
    logs = shadow_pipeline.logs()
    display(logs.loc[:,['out.variable','out_housingchallenger01.variable','out_housingchallenger02.variable']])
else:
    logs = shadow_pipeline.logs()
    display([(log.model_name, log.output) for log in logs])
    shadow_logs = shadow_pipeline.logs_shadow_deploy()
    display(shadow_logs)

,out.variable,out_housingchallenger01.variable,out_housingchallenger02.variable
0,[448627.72],[445568.9],[461039.66]
1,[1208638.0],[1103661.8],[1126173.8]
2,[437177.84],[421732.38],[403378.1]
3,[981676.6],[873592.25],[917436.4]
4,[435628.56],[391048.06],[359483.9]
...,...,...,...
95,[827411.0],[743487.94],[787589.25]
96,[441960.38],[381577.16],[411258.3]
97,[1060847.5],[1520770.0],[1491293.8]
98,[706823.56],[663008.75],[594914.2]


### Undeploy Shadow Pipeline

We can now undeploy the shadow deployed pipeline to return the resources back to the Wallaroo instance.

In [102]:
shadow_pipeline.undeploy()

name,housepriceshadowtesting
created,2023-03-14 17:03:17.045793+00:00
last_updated,2023-03-14 17:03:17.642834+00:00
deployed,False
tags,
versions,"b28d11a4-2f03-4d43-bfdf-7d5ffd570920, ff47eeff-510a-497a-b5f0-285d9d227d5a"
steps,housingcontrol


## Model Swap

Now that we've completed our testing, we can swap our deployed model in the original `housepricingpipeline` with one we feel works better.  This is one with the pipeline `replace_with_model_step` method, where we specify the pipeline step and the model to replace it with.  This pipeline had only one step with the `rf_model.onnx` model, and we'll swap it out with the `gbr_model.onnx` model.

The model swap capability makes updating a pipeline with new models a quick production process.  

We'll start with sending 1,000 separate inference requests to the pipeline while we swap the model in and out.  This way we can show updates to the log files, and how the inferences are performing in the Wallaroo Dashboard.

In [107]:
# inferences in the background

import multiprocessing

if arrowEnabled is True:
    def background_inferences():
        #Read in the test data as one dataframe
        test_data = pd.read_json('data/xtest-1k.df.json')
        # For each row, submit that row as a separate dataframe
        # Add the results to the responses array
        for index, row in test_data.head(1000).iterrows():
            mainpipeline.infer(row.to_frame('tensor').reset_index(drop=True))
else:
    def background_inferences():
        from data import test_data
        for nth in range(1000):
            mainpipeline.infer(test_data.data[nth])


In [108]:
multiprocessing.Process(target=background_inferences).start()

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/homebrew/anaconda3/envs/arrowtests/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/homebrew/anaconda3/envs/arrowtests/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'background_inferences' on <module '__main__' (built-in)>


In [104]:
# Swap the model with a new one, then redeploy the pipeline

mainpipeline.replace_with_model_step(0, housing_model_challenger01).deploy()

name,housepricingpipeline
created,2023-03-14 15:31:57.026034+00:00
last_updated,2023-03-14 17:27:17.271175+00:00
deployed,True
tags,
versions,"7569d299-d105-4e2d-92a1-ea9878ce80c2, 30aba9c8-e09e-42e0-b2d5-086168623c44, 7f92d873-6140-4639-9cd0-e3259a9d57c9, bb221c3e-61e6-4074-912a-c57ba6bf0d34, 39aa0a7e-2d6d-4849-a3d5-6726995ed7de, c2289074-2445-4e49-8ffd-b8ec4a8c8317, fc8561c5-f338-4c8e-a15b-b23ed11fd137"
steps,housingcontrol


In [105]:
# show log reults

display(mainpipeline.logs())

,time,in.tensor,out.variable,check_failures
0,2023-03-14 17:27:37.629,"[4.0, 2.5, 2900.0, 5505.0, 2.0, 0.0, 0.0, 3.0, 8.0, 2900.0, 0.0, 47.6063, -122.02, 2970.0, 5251.0, 12.0, 0.0, 0.0]",[659806.0],0
